## Nosso dataset :

In [ ]:
!wget https://github.com/otaviofcoletti/Sara_dataset/raw/main/Sara_dataset_direito.zip

In [1]:
!unzip Sara_dataset_direito.zip


Archive:  Sara_dataset_direito.zip
replace Sara_dataset_direito/Situação rolo 3 da esquerda levantado index 14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [59]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.convolution_based import Arsenal, RocketClassifier


import plotly.express as px
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

from aeon.classification.feature_based import Catch22Classifier



In [2]:
lista_de_arquivos = os.listdir("Sara_dataset_direito")

In [3]:
# Function to determine the unit based on the sensor type
def get_unit(sensor):
    if sensor == "Gyroscope":
        unit = "(deg/s)"
    else:
        unit = "(g)"
    return unit

In [4]:
label_map = {
        "Situação referencia index": 'Normal',
        "Situação rolo 3 da esquerda levantado index": 'Left Up',
        "Situação rolo 3 da esquerda removido index": 'Left Un.',
        "Situação rolo 3 da direita levantado index": 'Right Up',
        "Situação rolo 3 da direita removido index": 'Right Down',
    }

# Random Forest (dados brutos) 5 classes


In [100]:


# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Split the data into training and testing sets
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)




In [101]:
# Initialize and train the Random Forest model
random_forest = RandomForestClassifier()
random_forest.fit(X_treino, y_treino)
predictions = random_forest.predict(X_teste)

In [102]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.7692307692307693


In [103]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.8748180247172183


In [104]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('random_forest.html')


# Catch22 + Isolation Forest 2 classes

In [105]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Split the data into training and testing sets
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the Random Forest model



In [106]:
from aeon.transformations.collection.catch22 import Catch22

In [107]:


clf = Catch22Classifier()

clf.fit(X_treino, y_treino)
predictions = clf.predict(X_teste)

In [108]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7692307692307693


In [109]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.8866507335257335


In [110]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('catch22.html')


# HIVECOTE

In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Split the data into training and testing sets
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the Random Forest model



In [50]:
from aeon.classification.hybrid import HIVECOTEV2


In [51]:
# create model
hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)

hc2.fit(X_treino, y_treino)
predictions = hc2.predict(X_teste)



In [52]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7948717948717948


In [53]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('hivecote2.html')


In [54]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.8910774410774411


# Time Series Forest

In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Split the data into training and testing sets
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the Random Forest model



In [38]:
from pyts.datasets import load_gunpoint
from pyts.classification import TimeSeriesForest

clf = TimeSeriesForest(random_state=1)
clf.fit(X_treino, y_treino)
predictions = clf.predict(X_teste)

In [39]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8205128205128205


In [40]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('time_series_forest.html')


In [41]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.9037878787878789


# Multirocket Arsenal 

In [16]:
label_map = {
        "Situação referencia index": 'Normal',
        "Situação rolo 3 da esquerda levantado index": 'Left Up',
        "Situação rolo 3 da esquerda removido index": 'Left Un.',
        "Situação rolo 3 da direita levantado index": 'Right Up',
        "Situação rolo 3 da direita removido index": 'Right Down',
    }

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Split the data into training and testing sets
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize and train the Random Forest model



In [18]:
multi_r = Arsenal(rocket_transform="multirocket")
multi_r.fit(X_treino, y_treino)
predictions = multi_r.predict(X_teste)

In [19]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9230769230769231


In [28]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('multirocket.html')


In [31]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.965909090909091


# MiniRocket Arsenal

In [32]:
multi_r = Arsenal(rocket_transform="minirocket")
multi_r.fit(X_treino, y_treino)
predictions = multi_r.predict(X_teste)




In [33]:
accuracy = accuracy_score(y_teste, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8974358974358975


In [34]:
conf_matrix = confusion_matrix(y_teste, predictions)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('minirocket.html')


In [35]:
predictions = np.array(predictions)
y_teste = np.array(y_teste)

# Binarize the labels
y_binarized = label_binarize(y_teste, classes=np.unique(y_teste))

# Ensure pred is a 2D array with one column per class
predictions = predictions.reshape(-1, 1) if predictions.ndim == 1 else predictions

# Binarize the predicted labels
pred_binarized = label_binarize(predictions, classes=np.unique(y_teste))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.9545454545454545
